**Gradio app file**

In [ ]:
import tensorflow as tf
import numpy as np
import json
import gradio as gr
from PIL import Image

# Load model
model = tf.keras.models.load_model("food_vision_model.keras")

# Load metadata
with open("food_vision_metadata.json", "r") as f:
    food_info = json.load(f)

class_names = list(food_info.keys())

def load_and_prep_image(img, img_size=(224, 224)):
    img = img.resize(img_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    return img_array

def predict(img):
    img_array = load_and_prep_image(img)
    pred = model.predict(img_array)[0]
    pred_class = class_names[np.argmax(pred)]
    confidence = float(np.max(pred))
    info = food_info.get(pred_class, {})

    result = f"""🍽️ **Food**: {pred_class.replace('_', ' ').title()}
🌍 **Ethnicity**: {info.get('ethnicity', 'N/A')}
🥦 **Ingredients**: {info.get('ingredients', 'N/A')}
🧪 **Nutrients**:
  - Calories: {info.get('nutrients', {}).get('Calories', 'N/A')} kcal
  - Carbs: {info.get('nutrients', {}).get('Carbs', 'N/A')}g
  - Protein: {info.get('nutrients', {}).get('Protein', 'N/A')}g
  - Fat: {info.get('nutrients', {}).get('Fat', 'N/A')}g
❤️ **Health Advice**: {info.get('health', 'N/A')}
🌱 **Diet Type**: {info.get('diet', 'N/A')}
🔍 **Confidence**: {confidence:.2%}"""

    return result

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs="markdown",
    title="🍛 Nigerian Food Classifier",
    description="Upload a food image to predict and get rich food metadata (ingredients, nutrients, and more)."
)

demo.launch()
